In [ ]:
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def fft(x, invert):
    N = len(x)
    # base case
    if N == 1:
        return x
    
    # split in even and odd parts and recursively compute their fft
    X_even = fft(x[::2], invert)
    X_odd = fft(x[1::2], invert)

    # define dft coefficient
    ang = 2 * math.pi / N * (-1 if invert else 1)
    w = 1 + 0j
    wn = math.cos(ang) + 1j*math.sin(ang)
    X = [(0 + 0j) for _ in range(N)]

    # merge the even and odd parts to get solution for current subproblem
    for i in range(0,int(N/2)):
        X[i] = X_even[i] + w * X_odd[i]
        X[i + int(N/2)] = X_even[i] - w * X_odd[i]
        if invert:
            X[i] /= 2
            X[i + int(N/2)] /= 2
        w *= wn;
    return X


In [ ]:
def fftshift_image(img):
    rows,cols = len(img),len(img[0])
    new_img = [[0 for _ in range(cols)] for __ in range(rows)]
    for i in range(rows):
        for j in range(int(cols/2)):
            new_img[i][j] = img[i][j+int(cols/2)]
            new_img[i][j+int(cols/2)] = img[i][j]

    for j in range(cols):
        for i in range(int(rows/2)):
            tmp = new_img[i][j]
            new_img[i][j] = new_img[i+int(rows/2)][j]
            new_img[i+int(rows/2)][j] = tmp
    return new_img

In [ ]:
def pad_image(img):
    rows,cols = len(img),len(img[0])
    new_rows,new_cols = 1,1

    # get nearest higher power of 2 for row and col size
    while new_rows<rows:
        new_rows*=2
    while new_cols<cols:
        new_cols*=2

    new_img = [[0 for _ in range(new_cols)] for __ in range(new_rows)]

    for i in range(rows):
        for j in range(cols):
            new_img[i][j] = img[i][j]
    return new_img

def unpad(img,orig_rows,orig_cols):
    # crop the image to its original dimension
    new_img = [[img[i][j] for j in range(orig_cols)] for i in range(orig_rows)]
    return new_img

In [ ]:
def fft_image(img,invert):
    orig_rows,orig_cols = len(img),len(img[0])
    rows,cols = len(img),len(img[0])
    IMG = [[img[i][j] for j in range(cols)] for i in range(rows)]
   
    if invert:
        IMG = fftshift_image(IMG)
   
   # apply fft along columns
    for col in range(cols):
        cur = [IMG[row][col] for row in range(rows)]
        cur = fft(cur,invert)
        for row in range(rows):
            IMG[row][col] = cur[row]

    # apply fft along rows
    for row in range(rows):
        cur = [IMG[row][col] for col in range(cols)]
        cur = fft(cur,invert)
        for col in range(cols):
            IMG[row][col]=cur[col]
            
    if not invert:
        IMG = fftshift_image(IMG)
    return IMG

In [ ]:
def ideal_filter(f0,type,rows,cols):
    filter = [[0 for _ in range(cols)] for __ in range(rows)]
    for i in range(rows):
        for j in range(cols):
            d = math.sqrt(pow(i-int(rows/2),2)+pow(j-int(cols/2),2))
            if type=='low':
                if d<f0:
                    filter[i][j] = 1
            elif type=='high':
                if d>f0:
                    filter[i][j] = 1
    return filter

In [ ]:
def butterworth_filter(f0,type,rows,cols,order):
    filter = [[0 for _ in range(cols)] for __ in range(rows)]
    for i in range(rows):
        for j in range(cols):
            d = math.sqrt(pow(i-int(rows/2),2)+pow(j-int(cols/2),2))+1e-30
            if type=='low':
                filter[i][j] = 1/(1+(math.sqrt(2)-1)*pow(d/f0,2*order))
            elif type=='high':
                filter[i][j] = 1/(1+(math.sqrt(2)-1)*pow(f0/d,2*order)) 
    return filter

In [ ]:
def gaussian_filter(f0,type,rows,cols):
    filter = [[0 for _ in range(cols)] for __ in range(rows)]
    sigma = f0/(math.sqrt(math.log(2)))
    for i in range(rows):
        for j in range(cols):
            d = math.sqrt(pow(i-int(rows/2),2)+pow(j-int(cols/2),2))+1e-30
            if type=='low':
                filter[i][j] = math.exp(-1/(2*sigma*sigma)*d*d)
            elif type=='high':
                filter[i][j] = math.exp(-pow(f0,4)/(2*sigma*sigma*d*d)) 
    return filter

In [ ]:
def apply_filter(img,filter):
    rows, cols = len(img), len(img[0])
    new_img = [[img[i][j] for j in range(cols)] for i in range(rows)]

    # multiply elementwise to apply filter in freq domain
    for i in range(rows):
        for j in range(cols):
            new_img[i][j]*=filter[i][j]
    return new_img

In [ ]:
def read_image(file):
    img = cv2.imread("input/"+file)

    # convert returned numpy array to list
    
    img = img[:,:,0]
    img = [[complex(img[i][j]) for j in range(len(img[0]))] for i in range(len(img))]
    orig_rows,orig_cols = len(img),len(img[0])
    
    # pad image to make its dimension power of 2
    img = pad_image(img)
    return img,orig_rows,orig_cols

In [ ]:
def write_image(file, img, orig_rows, orig_cols):
    # remove added padding from image
    img = unpad(img,orig_rows,orig_cols)

    # scale pixels so as to lie from 0 to 255
    max_val = 0 
    for row in img:
        for pixel in row:
            max_val = max(max_val,pixel)
    img = [[img[i][j]/max_val*255 for j in range(orig_cols)] for i in range(orig_rows)]
    
    # convert list to numpy array for opencv to write
    img = np.array(img).astype(np.uint8)
    cv2.imwrite("output/"+file, img)

In [ ]:
def q1():
    # take all the inputs
    file = input("Enter file name: ")
    fltr = input("Input filter (ideal, gaussian, butterworth): ")
    typ = input("Enter type (high/low): ")
    f0 = int(input("Enter cutoff frequency: "))
    
    
    # read image and get fft
    img,orig_rows,orig_cols = read_image(file)
    rows,cols = len(img), len(img[0])
    IMG = fft_image(img,False)

    # get the filter matrix
    if fltr=='ideal':
        filter = ideal_filter(f0,typ,rows,cols)
    elif fltr=='gaussian':
        filter = gaussian_filter(f0,typ,rows, cols)
    elif fltr=='butterworth':
        order = int(input("Enter order: "))
        filter = butterworth_filter(f0,typ,rows,cols,order)
    else:
        print("invalid input")
        return
    
    # plot log magnitude spectrum of original image
    log_mag_spectrum = [[20*math.log10(1+abs(IMG[i][j])) for j in range(cols)] for i in range(rows)]
    write_image("original_spectrum.jpg", log_mag_spectrum,orig_rows,orig_cols)

    # plot filter's frequency response
    filter_freq_response = [[255*filter[i][j] for j in range(cols)] for i in range(rows)]
    write_image("freq_response.jpg", filter_freq_response,orig_rows,orig_cols)

    # plot filtered spectrum
    IMG = apply_filter(IMG,filter)
    log_mag_spectrum = [[20*math.log10(1+abs(IMG[i][j])) for j in range(cols)] for i in range(rows)]
    write_image("filtered_spectrum.jpg", log_mag_spectrum,orig_rows,orig_cols)

    # plot filtered image after taking abs of each pixel
    img_filtered = fft_image(IMG,True)
    img_filtered = [[abs(img_filtered[i][j]) for j in range(cols)] for i in range(rows)]
    write_image("filtered_image.jpg", img_filtered,orig_rows,orig_cols)

In [ ]:
def q2(file1="einstein.png", file2="marilyn.png"):
    # read both images
    img1,orig_rows,orig_cols = read_image(file1)
    img2,_,__ = read_image(file2)

    # get fft of first image and pass through highpass filter
    IMG1 = fft_image(img1,False)
    filter = np.array(gaussian_filter(25,'high',len(img1),len(img1[0]))).astype(np.complex64)
    IMG1 = apply_filter(IMG1,filter)

    # get fft of second image and pass through lowpass filter
    IMG2 = fft_image(img2,False)
    filter = np.array(gaussian_filter(20,'low',len(img2),len(img2[0]))).astype(np.complex64)
    IMG2 = apply_filter(IMG2,filter)

    # merge both images by elementwise addition
    IMG = [[0 for _ in range(len(IMG1[0]))] for __ in range(len(IMG1))]
    max_val = 0
    for i in range(len(IMG)):
        for j in range(len(IMG[0])):
            IMG[i][j] = IMG1[i][j]+IMG2[i][j]
    
    # take inverse fft and get output hybrid image
    img = fft_image(IMG,True)
    img = [[abs(img[i][j]) for j in range(len(img[0]))] for i in range(len(img))]
    write_image("hybrid.jpg", img,orig_rows,orig_cols)

In [ ]:
def q3():
    # get the input image
    file = input("Enter file name: ")
    img,orig_rows,orig_cols = read_image(file)
    rows,cols = len(img),len(img[0])
    
    # set the neighborhood dimension and threshold ratio
    r,th = 2,10

    # get fft of image
    IMG = fft_image(img, False)
    log_mag_spectrum = [[20*math.log10(1+abs(IMG[i][j])) for j in range(cols)] for i in range(rows)]
    write_image("original_spectrum.jpg", log_mag_spectrum,orig_rows,orig_cols)
    
    # apply frequency domain median filter on spectrum of image
    MED = [[0 for _ in range(cols)] for i in range(rows)]
    for i in range(rows):
        for j in range(cols):
            vals = []
            for k in range(-r,r+1):
                for l in range(-r,r+1):
                    if (i+k>=0) & (i+k<rows) & (j+l>=0) & (j+l<cols):
                        vals.append(IMG[i+k][j+l])
            vals.sort(key=abs)
            vals[int(len(vals)/2)]
            MED[i][j] = vals[int(len(vals)/2)]
    
    # update spectrum if ratio with median exceeds the threshold
    for i in range(rows):
        for j in range(cols):
            if abs(i-int(rows/2))<=r and abs(j-int(cols/2))<=r:
                continue
            if abs(IMG[i][j])/abs(MED[i][j])>th:
                IMG[i][j] = MED[i][j]
    
    # plot manipulated spectrum
    log_mag_spectrum = [[20*math.log10(1+abs(IMG[i][j])) for j in range(cols)] for i in range(rows)]
    write_image("filtered_spectrum.jpg", log_mag_spectrum,orig_rows,orig_cols)

    # get the resultant image and plot
    denoise_img = fft_image(IMG,True)
    denoise_img = [[abs(denoise_img[i][j]) for j in range(len(denoise_img[0]))] for i in range(len(denoise_img))]
    write_image("denoise.jpg",denoise_img,orig_rows,orig_cols)

In [53]:
q = int(input("Enter which part to test (1,2,3): "))
if q==1:
    q1()
elif q==2:
    q2()
elif q==3:
    q3()

Enter which part to test (1,2,3): 1
Enter file name: walkbridge.jpg
Input filter (ideal, gaussian, butterworth): butterworth
Enter type (high/low): high
Enter cutoff frequency: 30
Enter order: 2
